# (Step 2) - Initial setup of a software pipeline for deployment of a model that can be used in a browser



The following pipeline uses the Austrian Leaves dataset and sequential model as developed in **(Step 1) - Exploring image dataset Austrian Leaves** ([Colab:](https://colab.research.google.com/drive/15h5ILbqYFrsKk3sYKz5-e4Ln4zJs7HuS?usp=sharing), [Github](https://github.com/TechLabs-Berlin/st22-active-learn-trees/blob/main/DL-neural-network/Step1-exploring-datasets/(Step_1)_Exploring_image_dataset_Austrian_Leaves.ipynb))


The final model weights ([DL-neural-network/ai-model](https://github.com/TechLabs-Berlin/st22-active-learn-trees/blob/main//DL-neural-network/ai-model)), and a json file with the categories ([DL-neural-network/ai-model/class_definitions.json](https://github.com/TechLabs-Berlin/st22-active-learn-trees/blob/main/DL-neural-network/ai-model/class_definitions.json)) are stored on github and can be  accessed for browser side classification. 
The browser side implementation of the model is realised in an [html page -  (tree_identifying-update-links.html) ](https://github.com/TechLabs-Berlin/st22-active-learn-trees/blob/main/DL-neural-network/Step2-browserside-model-integration/tree_identifying-update-links.html) which shows how to upload an image, access the modelweights, classify the image and return the results, this api is now integrated by the WD team with React in our final web application.
When downloading the model weights with tensorflow.js not all types of NN layers are supproted, therefore the current model on web application is a very striped down version of the inital NN and therefore not giving accurate predictions. This should definetly be  improved but we thought it was more important to identify a solution to access the model on the browser side, without the need to fully set up a backend. 

The NN model will be improved in the following steps but still experimentally in a google colab environment. To be able to deploy an updated NN the difficutlties with tensorflow.js would need to be overcome in the future.


#### Setup Part 1

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [1]:
!curl -O https://zenodo.org/record/4446955/files/Leaves.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  135M  100  135M    0     0  9704k      0  0:00:14  0:00:14 --:--:-- 13.3M


In [2]:
!unzip -q Leaves.zip
!ls

Leaves	Leaves.zip  sample_data


In [3]:
!ls Leaves

 Ash   Beech   Hornbeam  'Mountain oak'  'Sycamore maple'


In [4]:
species = ['Ash' ,  'Beech'  , 'Hornbeam' , 'Mountain oak' , 'Sycamore maple']

In [5]:
len(species)

5

## Image dataset preparation for model training

In [7]:
args = {
    "labels": "inferred",
    "label_mode": "categorical",
    "batch_size": 32,
    "image_size": (256, 256),
    "seed": 1,
    "validation_split": .2,
    "class_names": species
}

In [8]:
train = tf.keras.utils.image_dataset_from_directory(
    "Leaves",
    subset="training",
    **args
)

test = tf.keras.utils.image_dataset_from_directory(
  "Leaves",
  subset="validation",
    **args
)

Found 122 files belonging to 5 classes.
Using 98 files for training.
Found 122 files belonging to 5 classes.
Using 24 files for validation.


## Train Sequential CNN model 

In [9]:
def train_model(network, epochs=5):
    model = Sequential(network)

    model.compile(
        optimizer='adam', # how to predict error/ tries to minimize error 
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
        )
    
    ## alternative Model compiling
    #model.compile(
    #    optimizer='rmsprop',
    #    loss='binary_crossentropy',
    #    metrics=['accuracy']
    #    )

    history = model.fit(
      train,
      validation_data=test,
      epochs=epochs
    )
    history_df = pd.DataFrame.from_dict(history.history)
    return history_df, model

In [10]:
network_1 = Sequential([
  tf.keras.layers.Rescaling(1./255), # rescale input layer
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(256,256,3)), # scan over image and generate new features
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(species))
])

In [11]:
history_df, model = train_model(network_1)

Epoch 1/5
4/4 [==============================] - 14s 354ms/step - loss: 130.3583 - accuracy: 0.2143 - val_loss: 4.4660 - val_accuracy: 0.7500
Epoch 2/5
4/4 [==============================] - 2s 270ms/step - loss: 28.9885 - accuracy: 0.3163 - val_loss: 88.2600 - val_accuracy: 0.4583
Epoch 3/5
4/4 [==============================] - 3s 281ms/step - loss: 33.6764 - accuracy: 0.5000 - val_loss: 25.8440 - val_accuracy: 0.4583
Epoch 4/5
4/4 [==============================] - 3s 405ms/step - loss: 4.8730 - accuracy: 0.6531 - val_loss: 9.7021 - val_accuracy: 0.6667
Epoch 5/5
4/4 [==============================] - 3s 279ms/step - loss: 11.1241 - accuracy: 0.5714 - val_loss: 8.6706 - val_accuracy: 0.6667


## Convert Keras Model to java script readable version using tensorflow js

### Setup Part 2

https://www.tensorflow.org/js/tutorials/conversion/import_keras

https://www.youtube.com/watch?v=dMq4nAMuqO8

In [ ]:
!pip install tensorflowjs

### Download model to tensorflow js

In [13]:
import tensorflowjs as tfjs

In [14]:
!mkdir /content/trees_identifying_model

In [15]:
tfjs.converters.save_keras_model(model, 'trees_identifying_model')

### create zip folder for download of model weights

In [16]:
!tar cf trees_identifying_js.tar /content/trees_identifying_model -- strip 1

tar: Removing leading `/' from member names
tar: strip: Cannot stat: No such file or directory
tar: 1: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [17]:
!gzip -9 trees_identifying_js.tar